In [1]:
import pandas as pd
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity


In [2]:
# Load the data 
df = pd.read_excel(io='Online_Retail.xlsx', sheet_name = 'Online Retail')

In [3]:
df.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom


In [4]:
# There are negative records in Quantity
df = df.loc[df['Quantity'] > 0]

### Data Preparation

* Handle Nan values
* Build a customer-to-item matrix

In [5]:
df.isna().sum()

InvoiceNo           0
StockCode           0
Description       592
Quantity            0
InvoiceDate         0
UnitPrice           0
CustomerID     133361
Country             0
dtype: int64

#### as we are building a customer item matrix each row defines one customer usage/purchase so deleting rows with customer id as null. we need to exclude them from analysis

In [6]:
df = df.dropna(subset = ['CustomerID'])

### Building a customer item matrix

In [7]:
customer_item_matrix = df.pivot_table(index='CustomerID', columns='StockCode', values = 'Quantity', aggfunc = 'sum')

In [8]:
customer_item_matrix.head()

StockCode,10002,10080,10120,10125,10133,10135,11001,15030,15034,15036,...,90214V,90214W,90214Y,90214Z,BANK CHARGES,C2,DOT,M,PADS,POST
CustomerID,,,,,,,,,,,,,,,,,,,,,
12346.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
12347.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
12348.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,9.0
12349.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0
12350.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0


### Now lets 0-1 encode this data, value of 1 means customer has purchased this data and 0 means the customer has never purchased this product.

In [9]:
customer_item_matrix = customer_item_matrix.applymap(lambda a: 1 if a > 0 else 0)

In [10]:
customer_item_matrix

StockCode,10002,10080,10120,10125,10133,10135,11001,15030,15034,15036,...,90214V,90214W,90214Y,90214Z,BANK CHARGES,C2,DOT,M,PADS,POST
CustomerID,,,,,,,,,,,,,,,,,,,,,
12346.0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
12347.0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
12348.0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
12349.0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
12350.0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18280.0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
18281.0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
18282.0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


### Building product recommender engines
* Collaborative filtering
* Approach 1: User based : we compute similarities between users based on the items they purchase
* Approach 2: Item based : we compute similarities between items based on which items are brought together with which other items

In [11]:
# The cosine similarity function in sklearn package computes the pair wise cosine similarity in given data

### User based collaborative filtering and recommendations

In [12]:
# compute cosing similarities between 2 users
user_user_sim_matrix = pd.DataFrame(cosine_similarity(customer_item_matrix))

In [13]:
user_user_sim_matrix.head()

,0,1,2,3,4,5,6,7,8,9,...,4329,4330,4331,4332,4333,4334,4335,4336,4337,4338
0,1.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,...,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.000000
1,0.0,1.000000,0.063022,0.046130,0.047795,0.038484,0.0,0.025876,0.136641,0.094742,...,0.0,0.029709,0.052668,0.0,0.032844,0.062318,0.0,0.113776,0.109364,0.012828
2,0.0,0.063022,1.000000,0.024953,0.051709,0.027756,0.0,0.027995,0.118262,0.146427,...,0.0,0.064282,0.113961,0.0,0.000000,0.000000,0.0,0.000000,0.170905,0.083269
3,0.0,0.046130,0.024953,1.000000,0.056773,0.137137,0.0,0.030737,0.032461,0.144692,...,0.0,0.105868,0.000000,0.0,0.039014,0.000000,0.0,0.067574,0.137124,0.030475
4,0.0,0.047795,0.051709,0.056773,1.000000,0.031575,0.0,0.000000,0.000000,0.033315,...,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.044866,0.000000


In [14]:
user_user_sim_matrix.columns = customer_item_matrix.index
user_user_sim_matrix['CustomerID'] = customer_item_matrix.index
user_user_sim_matrix = user_user_sim_matrix.set_index('CustomerID')

In [15]:
user_user_sim_matrix.head()

CustomerID,12346.0,12347.0,12348.0,12349.0,12350.0,12352.0,12353.0,12354.0,12355.0,12356.0,...,18273.0,18274.0,18276.0,18277.0,18278.0,18280.0,18281.0,18282.0,18283.0,18287.0
CustomerID,,,,,,,,,,,,,,,,,,,,,
12346.0,1.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,...,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.000000
12347.0,0.0,1.000000,0.063022,0.046130,0.047795,0.038484,0.0,0.025876,0.136641,0.094742,...,0.0,0.029709,0.052668,0.0,0.032844,0.062318,0.0,0.113776,0.109364,0.012828
12348.0,0.0,0.063022,1.000000,0.024953,0.051709,0.027756,0.0,0.027995,0.118262,0.146427,...,0.0,0.064282,0.113961,0.0,0.000000,0.000000,0.0,0.000000,0.170905,0.083269
12349.0,0.0,0.046130,0.024953,1.000000,0.056773,0.137137,0.0,0.030737,0.032461,0.144692,...,0.0,0.105868,0.000000,0.0,0.039014,0.000000,0.0,0.067574,0.137124,0.030475
12350.0,0.0,0.047795,0.051709,0.056773,1.000000,0.031575,0.0,0.000000,0.000000,0.033315,...,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.044866,0.000000


#### Product Recommendation

In [16]:
#rank most similar customer to customer with ID = 12350
user_user_sim_matrix.loc[12350.0].sort_values(ascending = False).head(10)
#Top 10 customer that are most similar to 12350 customer

CustomerID
12350.0    1.000000
17935.0    0.183340
12414.0    0.181902
12652.0    0.175035
16692.0    0.171499
16754.0    0.171499
12814.0    0.171499
12791.0    0.171499
16426.0    0.166968
16333.0    0.161690
Name: 12350.0, dtype: float64

### Lets pick a customer from above list of customers and build a strategy
* Strategy
* A = Customer B= Target customer
* Identify items that customer A and B have already bought.
* Find items that B has not purchased but A has purchased
* Since these 2 customers has brought similar items in past, we assume that target has high chance of purchasing what A has already purchased
* Used list of items and recommend to target customer

In [123]:
items_bought_by_A = set(customer_item_matrix.loc[12350.0].iloc[
customer_item_matrix.loc[12350.0].to_numpy().nonzero()
].index)

In [131]:
items_bought_by_B = set(customer_item_matrix.loc[17932.0].iloc[customer_item_matrix.loc[17935.0].to_numpy().nonzero()].index)

In [132]:
items_to_recommend_to_B = items_bought_by_A - items_bought_by_B
items_to_recommend_to_B

{20615,
 20652,
 21171,
 21832,
 21864,
 21908,
 21915,
 22348,
 22412,
 22620,
 '79066K',
 '79191C',
 '84086C'}

In [126]:
df.loc[df['StockCode'].isin(items_to_recommend_to_B),['StockCode','Description']].drop_duplicates().set_index('StockCode')

,Description
StockCode,
21832,CHOCOLATE CALCULATOR
21915,RED HARMONICA IN BOX
22620,4 TRADITIONAL SPINNING TOPS
79066K,RETRO MOD TRAY
21864,UNION JACK FLAG PASSPORT COVER
79191C,RETRO PLASTIC ELEPHANT TRAY
21908,CHOCOLATE THIS WAY METAL SIGN
20615,BLUE POLKADOT PASSPORT COVER
20652,BLUE POLKADOT LUGGAGE TAG


In [127]:
# How does cosine similarity work??

### Item based collaborative filtering and recommendation

In [22]:
item_item_sim_matrix = pd.DataFrame(cosine_similarity(customer_item_matrix.T))

In [23]:
item_item_sim_matrix.columns = customer_item_matrix.T.index
item_item_sim_matrix['CustomerID'] = customer_item_matrix.T.index
item_item_sim_matrix = item_item_sim_matrix.set_index('CustomerID')

In [24]:
item_item_sim_matrix

StockCode,10002,10080,10120,10125,10133,10135,11001,15030,15034,15036,...,90214V,90214W,90214Y,90214Z,BANK CHARGES,C2,DOT,M,PADS,POST
CustomerID,,,,,,,,,,,,,,,,,,,,,
10002,1.000000,0.000000,0.094868,0.090351,0.062932,0.098907,0.095346,0.047673,0.075593,0.090815,...,0.000000,0.0,0.000000,0.0,0.000000,0.029361,0.0,0.066915,0.000000,0.078217
10080,0.000000,1.000000,0.000000,0.032774,0.045655,0.047836,0.000000,0.000000,0.082261,0.049413,...,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.016182,0.000000,0.000000
10120,0.094868,0.000000,1.000000,0.057143,0.059702,0.041703,0.060302,0.060302,0.095618,0.028718,...,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.070535,0.000000,0.010993
10125,0.090351,0.032774,0.057143,1.000000,0.042644,0.044682,0.043073,0.000000,0.051224,0.030770,...,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.070535,0.000000,0.070669
10133,0.062932,0.045655,0.059702,0.042644,1.000000,0.280097,0.045002,0.060003,0.071358,0.057152,...,0.000000,0.0,0.000000,0.0,0.000000,0.036955,0.0,0.070185,0.049752,0.021877
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
C2,0.029361,0.000000,0.000000,0.000000,0.036955,0.019360,0.055989,0.000000,0.000000,0.039996,...,0.000000,0.0,0.000000,0.0,0.000000,1.000000,0.0,0.026196,0.000000,0.020413
DOT,0.000000,0.000000,0.000000,0.000000,0.000000,0.104257,0.150756,0.000000,0.000000,0.000000,...,0.000000,0.0,0.000000,0.0,0.000000,0.000000,1.0,0.000000,0.000000,0.000000
M,0.066915,0.016182,0.070535,0.070535,0.070185,0.066184,0.106335,0.063801,0.059013,0.086089,...,0.049875,0.0,0.040723,0.0,0.089220,0.026196,0.0,1.000000,0.000000,0.077539


### Strategy
* find most similar item
* recommend most similar item to customer

In [102]:
top_10_similar_items = list(item_item_sim_matrix.loc['PADS'].sort_values(ascending = False).iloc[:10].index)

In [103]:
top_10_similar_items

['PADS',
 84905,
 '46776D',
 '46776E',
 '84804B',
 '46776A',
 '46776F',
 85087,
 20835,
 '84800S']

In [104]:
df.loc[
    df['StockCode'].isin(top_10_similar_items),
    ['StockCode','Description']
].drop_duplicates().set_index('StockCode').loc[top_10_similar_items]

,Description
StockCode,
PADS,PADS TO MATCH ALL CUSHIONS
84905,PINK BUTTERFLY CUSHION COVER
46776D,WOVEN SUNSET CUSHION COVER
46776E,WOVEN CANDY CUSHION COVER
84804B,BLUE DELPHINIUM ARTIFICIAL FLOWER
46776A,WOVEN BUBBLE GUM CUSHION COVER
46776F,WOVEN ROSE GARDEN CUSHION COVER
85087,CANDY SPOT CUSHION COVER
20835,FRENCH LATTICE CUSHION COVER


In [105]:
top_10_similar_items = list(item_item_sim_matrix.loc['PADS'].sort_values(ascending = False).iloc[:10].index)

In [110]:
cust = df.CustomerID.astype(int).drop_duplicates()
cust.head()
c = cust.values.tolist()


In [109]:
c = df[df['CustomerID'].isin(item_item_sim_matrix)]

In [98]:
item_item_sim_matrix.tail()

StockCode,10002,10080,10120,10125,10133,10135,11001,15030,15034,15036,...,90214V,90214W,90214Y,90214Z,BANK CHARGES,C2,DOT,M,PADS,POST
CustomerID,,,,,,,,,,,,,,,,,,,,,
C2,0.029361,0.000000,0.000000,0.000000,0.036955,0.019360,0.055989,0.000000,0.000000,0.039996,...,0.000000,0.0,0.000000,0.0,0.000000,1.000000,0.0,0.026196,0.0,0.020413
DOT,0.000000,0.000000,0.000000,0.000000,0.000000,0.104257,0.150756,0.000000,0.000000,0.000000,...,0.000000,0.0,0.000000,0.0,0.000000,0.000000,1.0,0.000000,0.0,0.000000
M,0.066915,0.016182,0.070535,0.070535,0.070185,0.066184,0.106335,0.063801,0.059013,0.086089,...,0.049875,0.0,0.040723,0.0,0.089220,0.026196,0.0,1.000000,0.0,0.077539
PADS,0.000000,0.000000,0.000000,0.000000,0.049752,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.000000,1.0,0.000000
POST,0.078217,0.000000,0.010993,0.070669,0.021877,0.034383,0.058004,0.016573,0.026278,0.051301,...,0.038866,0.0,0.031734,0.0,0.017381,0.020413,0.0,0.077539,0.0,1.000000


In [118]:
for i in c:
    print(i)
    i

17850
13047
12583
13748
15100
15291
14688
17809
15311
16098
18074
17420
16029
16250
12431
17511
13705
13747
13408
13767
17924
13448
15862
15513
12791
16218
14045
14307
17908
17920
12838
13255
16583
18085
13758
13694
15983
14849
17968
16210
17897
17377
16552
17181
17951
14729
12748
15012
12868
17572
14078
14001
12662
15525
14237
17905
15485
12433
16955
15350
15605
18144
15922
14594
15165
14911
16456
17346
17643
17841
17873
13093
12921
13468
17760
16928
16048
16274
14496
14696
16539
17025
13777
17690
12947
17460
18229
14142
17069
13065
14606
16835
15235
13576
18011
13090
15694
14741
13715
17732
12855
15752
17855
14047
17925
13941
17017
14135
13108
15601
13418
15658
14388
14901
18041
15955
15070
16244
15111
14390
16546
15260
13305
14491
14060
15923
16752
17287
15363
12915
15544
15738
17381
15827
14180
13117
16916
17964
14466
17235
16510
17802
15107
17976
14449
16781
13491
16186
17685
17581
15732
13138
15823
17567
15061
16203
15640
15574
16770
17838
17228
14829
17412
14031
14775
12971
1765

17282
16434
17050
13171
17472
17255
16084
17667
12507
14631
13381
15236
13475
17829
13717
13157
14644
12993
17571
16979
16738
15122
17612
17348
13893
14035
13623
14346
16431
12616
16317
14935
16073
14477
14760
14897
15810
16509
14386
12455
14530
15670
12653
16126
14609
13414
12823
12352
12372
14944
16494
14959
16148
17422
17175
18257
12362
13018
13718
15382
14754
13151
12654
13085
15903
15785
18138
14379
14211
14371
13425
12852
15228
13012
16182
13489
12714
13904
15755
13932
13799
13951
16765
16566
12891
16466
16115
14162
13301
14803
13744
12988
17323
12707
18094
15129
17361
18155
15321
13477
15065
16980
15146
12365
13410
13044
16441
16014
14520
18087
17994
13038
17002
15722
16349
16913
17442
14895
18114
15303
12845
18188
16045
16573
13421
18196
13285
13184
18151
13804
17499
15397
14276
13453
18172
16394
17711
13865
16940
14289
16208
17063
13149
18068
15932
16692
17171
18231
14004
17139
14513
15974
17830
12565
12582
14231
17867
14880
13118
15121
17117
12945
18056
16072
16838
15543
1375

15992
15318
17914
16528
12442
16569
12650
14578
16000
15195
14087
14204
15471
13436
15520
13298
14569
12713
